In [48]:
import sys,os
import uproot4 as uproot
import awkward1 as ak
import json
import numpy as np
import matplotlib.pyplot as plt
from fcc_python_tools.locations import loc
from fcc_python_tools import kinematics
from particle import literals as lp
from fcc_python_tools import plotting

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)

file = uproot.open("/eos/experiment/fcc/ee/tmp/test.root")
tree = file['events']

Access the generated truth-level particles from `Particle`:

In [49]:
g_c = "Particle"
g = tree.arrays(filter_name=f"{g_c}*",how="zip")
g[g_c,'p'] = kinematics.calc_p(g,g_c)

Access the reco hadrons from `ReconstructedParticles`:

In [50]:
#Get all the variables in the tree
vars = tree.keys()

#Loacation of the reco particles
h_container = "ReconstructedParticles"
h_c = f'{h_container}/{h_container}'

#Keep the variables that aren't covMatrix
keep_vars = []
for v in vars:
    if("covMatrix" not in v and h_c in v):
        keep_vars.append(v)

h = tree.arrays(keep_vars,how="zip")
h[h_c,'p'] = kinematics.calc_p(h,h_c)

Access the particle to MC particle info for the charged hadrons:

In [51]:
h_toMC_c = "MCRecoAssociations#1"
h_toMC = tree.arrays(filter_name=f"{h_toMC_c}*",how="zip")

Associate the generated particle with the reco particle according to the index of `h_toMC`. Store the generated particle in `h["gen"]`:

In [52]:
h["gen"] = g[g_c][h_toMC[h_toMC_c,'index']]

Calculate difference in momentum between the reco and associated true particle:

In [53]:
h["delta_p"] = h[h_c,'p'] - h['gen','p']
#h[h_c,"delta_p_per"] = 100*(h[h_c,"delta_p"] / h['gen','p'])

#plotting.errorbar_hist(h[h_c],"delta_p_per","all_hadrons","$\delta(p)/p$","$\%$",-0.2,0.2,100)

ValueError: in ListOffsetArray64, cannot broadcast nested list

(https://github.com/scikit-hep/awkward-1.0/blob/0.3.1/src/cpu-kernels/operations.cpp#L778)